In [138]:
import pandas as pd

# Phase 1: Data cleaning

In [139]:
airline_raw = pd.read_csv("../data/raw/Airline Dataset.csv")

In [140]:
airline_raw.head()

,Passenger ID,First Name,Last Name,Gender,Age,Nationality,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents,Departure Date,Arrival Airport,Pilot Name,Flight Status
0,10856,Edithe,Leggis,Female,62,Japan,Coldfoot Airport,US,United States,NAM,North America,6/28/2022,CXF,Edithe Leggis,On Time
1,43872,Elwood,Catt,Male,62,Nicaragua,Kugluktuk Airport,CA,Canada,NAM,North America,12/26/2022,YCO,Elwood Catt,On Time
2,42633,Darby,Felgate,Male,67,Russia,Grenoble-Isère Airport,FR,France,EU,Europe,1/18/2022,GNB,Darby Felgate,On Time
3,78493,Dominica,Pyle,Female,71,China,Ottawa / Gatineau Airport,CA,Canada,NAM,North America,9/16/2022,YND,Dominica Pyle,Delayed
4,82072,Bay,Pencost,Male,21,China,Gillespie Field,US,United States,NAM,North America,2/25/2022,SEE,Bay Pencost,On Time


In [141]:
airline_raw.isna().sum()

Passenger ID            0
First Name              0
Last Name               0
Gender                  0
Age                     0
Nationality             0
Airport Name            0
Airport Country Code    0
Country Name            0
Airport Continent       0
Continents              0
Departure Date          0
Arrival Airport         0
Pilot Name              0
Flight Status           0
dtype: int64

In [142]:
airline_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98619 entries, 0 to 98618
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Passenger ID          98619 non-null  int64 
 1   First Name            98619 non-null  object
 2   Last Name             98619 non-null  object
 3   Gender                98619 non-null  object
 4   Age                   98619 non-null  int64 
 5   Nationality           98619 non-null  object
 6   Airport Name          98619 non-null  object
 7   Airport Country Code  98619 non-null  object
 8   Country Name          98619 non-null  object
 9   Airport Continent     98619 non-null  object
 10  Continents            98619 non-null  object
 11  Departure Date        98619 non-null  object
 12  Arrival Airport       98619 non-null  object
 13  Pilot Name            98619 non-null  object
 14  Flight Status         98619 non-null  object
dtypes: int64(2), object(13)
memory usage

# Phase 2: Data modeling


### Passenger table
    - Passenger ID (Primary Key)
    - First Name
    - Last Name
    - Gender
    - Age
    - Nationality

### Airport
    - Airport Name (Primary Key)
    - Airport Country Code
    - Country Name
    - Airport Continent

### Flight
    Flight ID (Primary Key)
    Departure Date
    Flight Status

### Flight Details
    Flight ID (Foreign Key)
    Passenger ID (Foreign Key)
    Arrival Airport (Foreign Key - references Airport)
    Pilot Name (Foreign Key - references Pilot)    

In [143]:
airline_raw.columns

Index(['Passenger ID', 'First Name', 'Last Name', 'Gender', 'Age',
       'Nationality', 'Airport Name', 'Airport Country Code', 'Country Name',
       'Airport Continent', 'Continents', 'Departure Date', 'Arrival Airport',
       'Pilot Name', 'Flight Status'],
      dtype='object')

In [144]:
# create passenger table 
columns = ['Passenger ID', 'First Name', 'Last Name', 'Gender', 'Age',
           'Nationality', 'Airport Name', 'Airport Country Code']
# passenger_raw = airline_raw[columns].sort_values(['Last Name']).head(10)
passenger_raw = airline_raw[columns].drop_duplicates()
passenger_raw.head()

,Passenger ID,First Name,Last Name,Gender,Age,Nationality,Airport Name,Airport Country Code
0,10856,Edithe,Leggis,Female,62,Japan,Coldfoot Airport,US
1,43872,Elwood,Catt,Male,62,Nicaragua,Kugluktuk Airport,CA
2,42633,Darby,Felgate,Male,67,Russia,Grenoble-Isère Airport,FR
3,78493,Dominica,Pyle,Female,71,China,Ottawa / Gatineau Airport,CA
4,82072,Bay,Pencost,Male,21,China,Gillespie Field,US


In [145]:
# create airport table
# I have to remove the duplicate
columns = ['Airport Name', 'Airport Country Code', 'Country Name',
           'Airport Continent', 'Continents']
airport_raw = airline_raw[columns].drop_duplicates()
airport_raw.head()

,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents
0,Coldfoot Airport,US,United States,NAM,North America
1,Kugluktuk Airport,CA,Canada,NAM,North America
2,Grenoble-Isère Airport,FR,France,EU,Europe
3,Ottawa / Gatineau Airport,CA,Canada,NAM,North America
4,Gillespie Field,US,United States,NAM,North America


In [146]:
airport_raw['Airport Name'].value_counts().head(10)

Airport Name
San Pedro Airport                  4
Newcastle Airport                  3
San Fernando Airport               3
Santa Maria Airport                3
Wau Airport                        2
Las Brujas Airport                 2
Puerto Rico Airport                2
Melbourne International Airport    2
St George Airport                  2
San Javier Airport                 2
Name: count, dtype: int64

In [147]:
# airport_raw[airport_raw['Airport Name'] == 'San Pedro Airport']
# airport_raw[airport_raw['Airport Name'] == 'Santa Maria Airport']
# airport_raw[airport_raw['Airport Name'] == 'Newcastle Airport']
airport_raw[airport_raw['Airport Name'] == 'San Javier Airport']

,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents
2266,San Javier Airport,ES,Spain,EU,Europe
20057,San Javier Airport,BO,"Bolivia, Plurinational State of",SAM,South America


In [148]:
# remove bad data from airport.

# List of airports and their correct details
correct_airports = [
    {
        'Airport Name': 'San Pedro Airport',
        'Airport Country Code': 'BZ',
        'Country Name': 'Belize',
        'Airport Continent': 'NAM',
        'Continents': 'North America'
    },
    {
        'Airport Name': 'San Javier Airport',
        'Airport Country Code': 'ES',
        'Country Name': 'Spain',
        'Airport Continent': 'EU',
        'Continents': 'Europe'
    }
]

# Loop through each airport and its correct details
for airport in correct_airports:
    airport_raw = airport_raw[(airport_raw['Airport Name'] != airport['Airport Name']) | 
                              ((airport_raw['Airport Name'] == airport['Airport Name']) & 
                               (airport_raw['Airport Country Code'] == airport['Airport Country Code']) & 
                               (airport_raw['Country Name'] == airport['Country Name']) & 
                               (airport_raw['Airport Continent'] == airport['Airport Continent']) & 
                               (airport_raw['Continents'] == airport['Continents']))]

airport_raw.shape

(9103, 5)

In [149]:
# add the ids to the airport

# Check if the combination of 'Airport Name' and 'Airport Country Code' is unique
# unique_combo = airport_raw['Airport Name'] + '-' + airport_raw['Airport Country Code']
unique_combo = airport_raw[['Airport Name', 'Airport Country Code']].apply('-'.join, axis=1)
is_unique_combo = unique_combo.is_unique

# Add IDs based on the uniqueness
if is_unique_combo:
    airport_raw['Airport_ID'] = range(1, len(airport_raw) + 1)
else:
    # If there are duplicate combinations, create a unique ID for each 'Airport Name'
    unique_airports = airline_raw['Airport Name'].unique()
    airport_ids = { name:idx for idx, name in enumerate(unique_airports, 1)}
    # I need to fix this - there is a bug there
    airport_raw['Airport_ID'] = airport_raw['Airport Name'].map(airport_ids)

In [150]:
airport_raw[airport_raw['Airport Name'] == 'Santa Maria Airport']

,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents,Airport_ID
496,Santa Maria Airport,BR,Brazil,SAM,South America,488
10650,Santa Maria Airport,PE,Peru,SAM,South America,6256
20909,Santa Maria Airport,PT,Portugal,EU,Europe,8164


In [151]:
airline_raw.columns

Index(['Passenger ID', 'First Name', 'Last Name', 'Gender', 'Age',
       'Nationality', 'Airport Name', 'Airport Country Code', 'Country Name',
       'Airport Continent', 'Continents', 'Departure Date', 'Arrival Airport',
       'Pilot Name', 'Flight Status'],
      dtype='object')

In [152]:
# create flight table
# I have to remove the duplicate
columns = ['Airport Name','Airport Country Code','Departure Date', 'Arrival Airport',
           'Pilot Name', 'Flight Status']
flight_raw = airline_raw[columns]
flight_raw.head()

,Airport Name,Airport Country Code,Departure Date,Arrival Airport,Pilot Name,Flight Status
0,Coldfoot Airport,US,6/28/2022,CXF,Edithe Leggis,On Time
1,Kugluktuk Airport,CA,12/26/2022,YCO,Elwood Catt,On Time
2,Grenoble-Isère Airport,FR,1/18/2022,GNB,Darby Felgate,On Time
3,Ottawa / Gatineau Airport,CA,9/16/2022,YND,Dominica Pyle,Delayed
4,Gillespie Field,US,2/25/2022,SEE,Bay Pencost,On Time


In [153]:
import pandas as pd
import hashlib

# Create unique_combo using join
unique_combo = flight_raw[['Airport Name', 'Arrival Airport', 'Pilot Name', 'Flight Status']].apply('-'.join, axis=1)
is_unique_combo = unique_combo.is_unique

# Add IDs based on the uniqueness
if is_unique_combo:
    flight_raw['Flight_ID'] = range(1, len(flight_raw) + 1)
else:
    # Generate unique IDs using a hash function
    flight_raw['Flight_ID'] = [int(hashlib.sha256(val.encode()).hexdigest(), 16) % 10**8 for val in unique_combo]


/var/folders/21/f45fbj896dg1czx0x93kxrl80000gn/T/ipykernel_24959/1842472168.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_raw['Flight_ID'] = range(1, len(flight_raw) + 1)


In [154]:
flight_raw.head()

,Airport Name,Airport Country Code,Departure Date,Arrival Airport,Pilot Name,Flight Status,Flight_ID
0,Coldfoot Airport,US,6/28/2022,CXF,Edithe Leggis,On Time,1
1,Kugluktuk Airport,CA,12/26/2022,YCO,Elwood Catt,On Time,2
2,Grenoble-Isère Airport,FR,1/18/2022,GNB,Darby Felgate,On Time,3
3,Ottawa / Gatineau Airport,CA,9/16/2022,YND,Dominica Pyle,Delayed,4
4,Gillespie Field,US,2/25/2022,SEE,Bay Pencost,On Time,5


In [155]:
flight_raw[flight_raw['Pilot Name'] == 'Darby Felgate']

,Airport Name,Airport Country Code,Departure Date,Arrival Airport,Pilot Name,Flight Status,Flight_ID
2,Grenoble-Isère Airport,FR,1/18/2022,GNB,Darby Felgate,On Time,3


## Phase 2.1 : Put it algather

In [167]:
merged_flight_airport = pd.merge(flight_raw, airport_raw, on=['Airport Name', 'Airport Country Code'])
final_merged_table = pd.merge(merged_flight_airport, passenger_raw, on=['Airport Name', 'Airport Country Code'])

final_merged_table.columns

Index(['Airport Name', 'Airport Country Code', 'Departure Date',
       'Arrival Airport', 'Pilot Name', 'Flight Status', 'Flight_ID',
       'Country Name', 'Airport Continent', 'Continents', 'Airport_ID',
       'Passenger ID', 'First Name', 'Last Name', 'Gender', 'Age',
       'Nationality'],
      dtype='object')

In [168]:
final_merged_table.head()

,Airport Name,Airport Country Code,Departure Date,Arrival Airport,Pilot Name,Flight Status,Flight_ID,Country Name,Airport Continent,Continents,Airport_ID,Passenger ID,First Name,Last Name,Gender,Age,Nationality
0,Coldfoot Airport,US,6/28/2022,CXF,Edithe Leggis,On Time,1,United States,NAM,North America,1,10856,Edithe,Leggis,Female,62,Japan
1,Coldfoot Airport,US,6/28/2022,CXF,Edithe Leggis,On Time,1,United States,NAM,North America,1,35918,Lara,Feige,Female,4,Philippines
2,Coldfoot Airport,US,6/28/2022,CXF,Edithe Leggis,On Time,1,United States,NAM,North America,1,63276,Lexi,Roser,Female,37,Colombia
3,Coldfoot Airport,US,6/28/2022,CXF,Edithe Leggis,On Time,1,United States,NAM,North America,1,78972,Leeland,Loveland,Male,39,Panama
4,Coldfoot Airport,US,6/28/2022,CXF,Edithe Leggis,On Time,1,United States,NAM,North America,1,86809,Patricio,Swainger,Male,10,Portugal


## Phase 2.2: do data modeling with ids

In [169]:
final_merged_table.columns

Index(['Airport Name', 'Airport Country Code', 'Departure Date',
       'Arrival Airport', 'Pilot Name', 'Flight Status', 'Flight_ID',
       'Country Name', 'Airport Continent', 'Continents', 'Airport_ID',
       'Passenger ID', 'First Name', 'Last Name', 'Gender', 'Age',
       'Nationality'],
      dtype='object')

In [175]:
columns = ['Passenger ID', 'First Name', 'Last Name', 'Gender', 'Age','Nationality']
passenger_raw_final = final_merged_table[columns]
passenger_raw_final.head()

,Passenger ID,First Name,Last Name,Gender,Age,Nationality
0,10856,Edithe,Leggis,Female,62,Japan
1,35918,Lara,Feige,Female,4,Philippines
2,63276,Lexi,Roser,Female,37,Colombia
3,78972,Leeland,Loveland,Male,39,Panama
4,86809,Patricio,Swainger,Male,10,Portugal


In [181]:
columns = ['Airport_ID','Airport Name', 'Airport Country Code', 'Country Name', 'Airport Continent','Continents']
airline_raw_final = final_merged_table[columns]
airline_raw_final.head()

,Airport_ID,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents
0,1,Coldfoot Airport,US,United States,NAM,North America
1,1,Coldfoot Airport,US,United States,NAM,North America
2,1,Coldfoot Airport,US,United States,NAM,North America
3,1,Coldfoot Airport,US,United States,NAM,North America
4,1,Coldfoot Airport,US,United States,NAM,North America


In [177]:
# airline_raw_final[airline_raw_final['Airport Name'] == 'Coldfoot Airport']

In [187]:
columns = ['Flight_ID','Departure Date','Arrival Airport', 'Pilot Name', 'Flight Status', 
       'Country Name', 'Airport_ID','Passenger ID']
flight_raw_final = final_merged_table[columns]
flight_raw_final.head()

,Flight_ID,Departure Date,Arrival Airport,Pilot Name,Flight Status,Country Name,Airport_ID,Passenger ID
0,1,6/28/2022,CXF,Edithe Leggis,On Time,United States,1,10856
1,1,6/28/2022,CXF,Edithe Leggis,On Time,United States,1,35918
2,1,6/28/2022,CXF,Edithe Leggis,On Time,United States,1,63276
3,1,6/28/2022,CXF,Edithe Leggis,On Time,United States,1,78972
4,1,6/28/2022,CXF,Edithe Leggis,On Time,United States,1,86809
